<a href="https://colab.research.google.com/github/juanfranbrv/curso-langchain/blob/main/RunnableParallel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. `RunnableParallel`**
---
RunnableParallel es un componente fundamental en LangChain que permite ejecutar múltiples runnables de manera paralela y combinar sus resultados en un diccionario. Su principal característica es la capacidad de procesar diferentes tareas simultáneamente, mejorando la eficiencia de los flujos de procesamiento.  

Las cadenas paralelas permiten ejecutar varias tareas simultáneamente, lo que reduce el tiempo de ejecución general

RunnableMap es un "sinonimo" de RunnableParallel.

- Ejecutan múltiples runnables concurrentemente  
- Reciben un diccionario de runnables  
- Devuelven un diccionario con los resultados


# **Preparando el entorno del cuaderno**
---
Configuramos el entorno de trabajo para utilizar LangChain con distintos modelos de lenguaje (LLMs).

- Obtenemos las claves API para acceder a los servicios de OpenAI, Groq, Google Hugging Face, Mistral, Together y Anthropic

- Instalamos la librería LangChain y las integraciones necesarias para cada uno de estos proveedores.

- Importamos las clases específicas de LangChain que permiten crear plantillas de prompts e interactuar con los diferentes modelos de lenguaje, dejándolo todo listo para empezar a desarrollar aplicaciones basadas en LLMs. (Este codigo se explico con detalle en el primer cuaderno)

Comenta (#) las librerias y modelos que no desees usar.
El uso de las API de OpenAI y Anthropic es de pago. El resto son gratuitas y para usarlas basta con registrarse y generar una API Key.  

En el primer cuaderno encontraras los enlaces a estos servicios y este codigo explicado

In [1]:
%%capture --no-stderr

# Importar la librería `userdata` de Google Colab.
# Esta librería se utiliza para acceder a datos de usuario almacenados de forma segura en el entorno de Colab.
from google.colab import userdata

# Obtener las claves API de diferentes servicios desde el almacenamiento seguro de Colab.
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')
GROQ_API_KEY=userdata.get('GROQ_API_KEY')
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
HUGGINGFACEHUB_API_TOKEN=userdata.get('HUGGINGFACEHUB_API_TOKEN')
MISTRAL_API_KEY=userdata.get('MISTRAL_API_KEY')
TOGETHER_API_KEY=userdata.get('TOGETHER_API_KEY')
ANTHROPIC_API_KEY=userdata.get('ANTHROPIC_API_KEY')
DEEPSEEK_API_KEY=userdata.get('DEEPSEEK_API_KEY')


# Instalar las librerías necesarias usando pip.
# El flag `-qU` instala en modo silencioso (`-q`) y actualiza las librerías si ya están instaladas (`-U`).
%pip install langchain -qU  # Instalar la librería principal de LangChain.


# Instalar las integraciones de LangChain con diferentes proveedores de LLMs.
%pip install langchain-openai -qU
%pip install langchain-groq -qU
%pip install langchain-google-genai -qU
%pip install langchain-huggingface -qU
%pip install langchain_mistralai -qU
%pip install langchain-together -qU
%pip install langchain-anthropic -qU

# Importar las clases necesarias de LangChain para crear plantillas de prompt.
# `ChatPromptTemplate` es la clase base para plantillas de chat.
# `SystemMessagePromptTemplate` se usa para mensajes del sistema (instrucciones iniciales).
# `HumanMessagePromptTemplate` se usa para mensajes del usuario.
from langchain.prompts import PromptTemplate, ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

# Importar las clases para interactuar con los diferentes LLMs a través de LangChain.
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_huggingface import HuggingFaceEndpoint
from langchain_mistralai import ChatMistralAI
from langchain_together import ChatTogether
from langchain_anthropic import ChatAnthropic


# Importamos la libreria para formatear mejor la salida
from IPython.display import Markdown, display

Ejemplo 1:
---
-   Cada función se ejecuta en paralelo
-   El resultado es un diccionario con las claves especificadas
-   No importa el orden de ejecución, ambas se procesan simultáneamente

In [2]:
from langchain_core.runnables import RunnableParallel, RunnableLambda

# Funciones simples para demostrar paralelismo
def duplicar(x):
    return x * 2

def sumar_cinco(x):
    return x + 5

# Crear un RunnableParallel que ejecuta ambas operaciones en paralelo
parallel_runnable = RunnableParallel(
    duplicado=RunnableLambda(duplicar),
    suma=RunnableLambda(sumar_cinco)
)

# Invocar con un valor
resultado = parallel_runnable.invoke(10)
print(resultado)
# Salida esperada: {'duplicado': 20, 'suma': 15}

{'duplicado': 20, 'suma': 15}


## **Ejemplo 2:**

---

In [10]:
from langchain_core.runnables import RunnableParallel, RunnableLambda


titulo_prompt_template = PromptTemplate.from_template(template="Genera un titulo para este texto: {texto}")

palabras_clave_prompt_template = PromptTemplate.from_template(template="Genera una lista de 5 palabras clave a partir de este texto: {texto}")

llm_gpt4omini = ChatOpenAI(model="gpt-4o-mini",api_key=OPENAI_API_KEY)

def generar_titulo(texto):
    return llm_gpt4omini.invoke(titulo_prompt_template.format(texto=texto))

def generar_palabras_clave(texto):
    return llm_gpt4omini.invoke(palabras_clave_prompt_template.format(texto=texto))

tareas = RunnableParallel(
                          titulo = RunnableLambda(lambda x: generar_titulo(x)),
                          palabras_clave = RunnableLambda(lambda x: generar_palabras_clave(x))
                          )

texto = '''
Eric Arthur Blair (Motihari, Raj Británico, 25 de junio de 1903-Londres, 21 de enero de 1950),1​2​ conocido por su seudónimo de George Orwell, fue un novelista, periodista, ensayista y crítico británico nacido en la India, autor entre otras obras de las novelas distópicas Rebelión en la granja (1945) y 1984 (1949).
Su obra lleva la marca de las experiencias autobiográficas vividas por el autor en tres etapas de su vida: su posición en contra del imperialismo británico que lo llevó al compromiso como representante de las fuerzas del orden colonial en Birmania durante su juventud; a favor del socialismo democrático, después de haber observado y sufrido las condiciones de vida de las clases sociales de los trabajadores de Londres y París; y en contra de los totalitarismos nazi y estalinista tras su participación en la guerra civil española, en el bando republicano.
Además de cronista, crítico de literatura y novelista, es uno de los ensayistas en lengua inglesa más destacados de las décadas de 1930 y de 1940. También es conocido por sus críticas al totalitarismo en su novela corta alegórica Rebelión en la granja (1945) y su novela distópica 1984 (1949), escrita en sus últimos años de vida y publicada poco antes de su fallecimiento, y en la que crea el concepto de «Gran Hermano», que desde entonces pasó al lenguaje común de la crítica de las técnicas modernas de vigilancia.
En 2008 figuraba en el puesto número dos del listado de los cincuenta escritores británicos de mayor relevancia desde 1945, elaborado por The Times.El adjetivo «orwelliano» es frecuentemente utilizado en referencia al distópico universo totalitario imaginado por el escritor británico.
'''

resultado=tareas.invoke({"texto":texto})

# Resultado es un diccionario con las dos claves de la tareas paralalelas
# print(resultado)

print(resultado["titulo"].content)
print("\n")
print(resultado["palabras_clave"].content)


"George Orwell: La Voz Crítica de una Era Distópica y su Lucha contra el Totalitarismo"


1. George Orwell  
2. Distopía  
3. Totalitarismo  
4. Imperialismo  
5. Socialismo


poner aqui ejemplo de como fusionar el resultado de  2 ramas en 1

Poner aqui el ejemplo de como fusionar N ramas en 1

A continuación encontrarás un ejemplo completo en el que:

Tres modelos (simulados) generan un texto cada uno a partir de la misma entrada.
Un cuarto modelo (simulado) toma esos tres textos y los “refunde” (los combina en un resultado final unificado).
Para simplificar, usaremos PromptTemplate y RunnableLambda en lugar de un LLM real, pero la estructura sería la misma si reemplazas las partes simuladas por un modelo real (por ejemplo,

Esta en chatgpt